In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check GPU type
!nvidia-smi

Sun Nov 10 00:53:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Install ultralytics
!pip -q install  ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 16.7 MB/s eta 0:00:00


In [4]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Path to where your data is stored
DATA_DIR = Path ('/content/drive/MyDrive/ghana_project')

# Preview data files available
os.listdir(DATA_DIR)

['images.zip',
 'Test.csv',
 'SampleSubmission.csv',
 'manifest-8517f262e2ca61a7f7ac8fc1d32eb0be20241003-7123-1lvxb66.json',
 'Train.csv',
 'Rail_Challenge_Starter.ipynb',
 'team_ghana_model.ipynb',
 'datasets',
 'balanced_model',
 'balanced_model2',
 'data.yaml']

In [6]:
# Set up directories for training a yolo model

# Images directories
DATASET_DIR = Path('/content/drive/MyDrive/ghana_project/datasets/dataset')
IMAGES_DIR = DATASET_DIR / 'images'
TRAIN_IMAGES_DIR = IMAGES_DIR / 'train'
VAL_IMAGES_DIR = IMAGES_DIR / 'val'
TEST_IMAGES_DIR = IMAGES_DIR / 'test'

# Labels directories
LABELS_DIR = DATASET_DIR / 'labels'
TRAIN_LABELS_DIR = LABELS_DIR / 'train'
VAL_LABELS_DIR = LABELS_DIR / 'val'
TEST_LABELS_DIR = LABELS_DIR / 'test'

In [ ]:
# Unzip images to 'images' dir
shutil.unpack_archive(DATA_DIR / 'images.zip', DATASET_DIR /'images')

In [7]:
# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

train['image_path'] = [DATASET_DIR / 'images' / x for x in train.Image_ID]
test['image_path'] = [DATASET_DIR / 'images' / x for x in test.Image_ID]
# Map str classes to ints (label encoding targets)
class_mapper = {x:y for x,y in zip(sorted(train['class'].unique().tolist()), range(train['class'].nunique()))}
train['class_id'] = train['class'].map(class_mapper)

# Preview the head of the train set
train.head()

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,image_path,class_id
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/content/drive/MyDrive/ghana_project/datasets/...,5
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/content/drive/MyDrive/ghana_project/datasets/...,5
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/content/drive/MyDrive/ghana_project/datasets/...,5
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/content/drive/MyDrive/ghana_project/datasets/...,5
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,/content/drive/MyDrive/ghana_project/datasets/...,8


In [8]:

test.head()

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,image_path
0,id_128pxx.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
1,id_12jbci.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
2,id_143s4o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
3,id_14tfmb.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
4,id_14tw4o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...


In [9]:

ss.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_128pxx.jpg,Corn_Cercospora_Leaf_Spot,0.5,100,100,100,100
1,id_128pxx.jpg,Corn_Common_Rust,0.5,100,100,100,100
2,id_128pxx.jpg,Corn_Healthy,0.5,100,100,100,100
3,id_128pxx.jpg,Corn_Northern_Leaf_Blight,0.5,100,100,100,100
4,id_128pxx.jpg,Corn_Streak,0.5,100,100,100,100


In [10]:
# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset = ['Image_ID'], ignore_index = True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size = 0.25, stratify=train_unique_imgs_df['class'], random_state=42) #changed random state

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check shapes of training and validation data
X_train.shape, X_val.shape

((30777, 9), (10252, 9))

In [ ]:
# Check if dirs exist, if they do, remove them, otherwise create them.
# This only needs to run once
for DIR in [TRAIN_IMAGES_DIR,VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR,VAL_LABELS_DIR,TEST_LABELS_DIR]:
  if DIR.exists():
    shutil.rmtree(DIR)
  DIR.mkdir(parents=True, exist_ok = True)

In [11]:
test.head()

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,image_path
0,id_128pxx.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
1,id_12jbci.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
2,id_143s4o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
3,id_14tfmb.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...
4,id_14tw4o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/ghana_project/datasets/...


In [12]:

# Preview target distribution, seems there a class imbalance that needs to be handled
X_train['class'].value_counts(normalize = True), X_val['class'].value_counts(normalize = True)

(class
 Corn_Cercospora_Leaf_Spot    0.160444
 Tomato_Septoria              0.159047
 Tomato_Late_Blight           0.098905
 Corn_Streak                  0.077201
 Tomato_Healthy               0.069045
 Pepper_Septoria              0.051922
 Pepper_Leaf_Mosaic           0.051662
 Tomato_Early_Blight          0.047763
 Pepper_Bacterial_Spot        0.047665
 Corn_Common_Rust             0.040290
 Pepper_Leaf_Curl             0.037561
 Corn_Healthy                 0.037561
 Tomato_Fusarium              0.019950
 Pepper_Healthy               0.017935
 Pepper_Late_Blight           0.014296
 Pepper_Leaf_Blight           0.012574
 Tomato_Bacterial_Spot        0.011860
 Pepper_Cercospora            0.011405
 Pepper_Fusarium              0.011340
 Tomato_Leaf_Curl             0.011177
 Corn_Northern_Leaf_Blight    0.004289
 Tomato_Mosaic                0.003314
 Pepper_Early_Blight          0.002794
 Name: proportion, dtype: float64,
 class
 Corn_Cercospora_Leaf_Spot    0.156067
 Tomato_Septori

In [ ]:
# Copy train, val and test images to their respective dirs
for img in tqdm(X_train.image_path.unique()):
  shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])

for img in tqdm(X_val.image_path.unique()):
  shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])

for img in tqdm(test.image_path.unique()):
  shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])

# Verify image counts
print(f"\nNumber of images in train: {len(list(TRAIN_IMAGES_DIR.glob('*')))})")
print(f"Number of images in val: {len(list(VAL_IMAGES_DIR.glob('*')))})")
print(f"Number of images in test: {len(list(TEST_IMAGES_DIR.glob('*')))})")

  0%|          | 0/3676 [00:00<?, ?it/s]

  0%|          | 0/1226 [00:00<?, ?it/s]

  0%|          | 0/2101 [00:00<?, ?it/s]


Number of images in train: 3676)
Number of images in val: 1226)
Number of images in test: 2101)


In [13]:
X_train.head()

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,image_path,class_id
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/content/drive/MyDrive/ghana_project/datasets/...,5
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/content/drive/MyDrive/ghana_project/datasets/...,5
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/content/drive/MyDrive/ghana_project/datasets/...,5
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/content/drive/MyDrive/ghana_project/datasets/...,5
6,id_11gglx.jpg,1.0,Corn_Cercospora_Leaf_Spot,774.562500,2735.933839,850.476742,2834.348725,/content/drive/MyDrive/ghana_project/datasets/...,0


In [14]:
!pip install -U albumentations opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.1/605.1 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [ ]:
import albumentations as A
import cv2
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import shutil

def load_image(image_path):
    """Load an image from path."""
    return cv2.imread(str(image_path))

def save_image(image, path):
    """Save an image to path."""
    cv2.imwrite(str(path), image)

# Define augmentation pipeline with modern transforms
transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, p=0.5),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=0.5),
    ], p=0.3),
    A.OneOf([
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
    ], p=0.3),
])

def augment_underrepresented_classes(X_train, min_percentage=0.05):
    """
    Augment classes that are below the minimum percentage threshold.

    Args:
        X_train: DataFrame containing image paths and classes
        min_percentage: Minimum percentage threshold (default: 5%)
    """
    # Calculate total samples
    total_samples = len(X_train)

    # Get class counts and percentages
    class_counts = X_train['class'].value_counts()
    class_percentages = X_train['class'].value_counts(normalize=True)

    # Calculate target samples for minimum percentage
    target_samples = int(total_samples * min_percentage)

    print(f"Minimum target samples per class (5%): {target_samples}")

    # Create new rows for augmented images
    new_rows = []

    # For each class below threshold
    for class_name in tqdm(class_percentages[class_percentages < min_percentage].index):
        current_count = class_counts[class_name]
        needed_samples = target_samples - current_count

        if needed_samples <= 0:
            continue

        # Get all images of current class
        class_images = X_train[X_train['class'] == class_name]

        # Calculate augmentations needed per image
        augmentations_per_image = int(np.ceil(needed_samples / len(class_images)))

        # For each image in the class
        for idx, row in class_images.iterrows():
            image_path = row['image_path']

            # Load image
            image = load_image(image_path)
            if image is None:
                print(f"Failed to load image: {image_path}")
                continue

            # Generate augmentations
            for i in range(augmentations_per_image):
                if len(new_rows) >= needed_samples:
                    break

                # Apply augmentation
                augmented = transform(image=image)

                # Create new image filename
                original_filename = Path(image_path).name
                new_filename = f"aug_{class_name}_{i}_{original_filename}"
                new_path = Path(TRAIN_IMAGES_DIR) / new_filename

                # Save augmented image
                save_image(augmented['image'], new_path)

                # Create new row with same bounding box coordinates
                new_row = row.copy()
                new_row['Image_ID'] = new_filename
                new_row['image_path'] = new_path
                new_rows.append(new_row)

                if len(new_rows) >= needed_samples:
                    break

    # Create DataFrame with augmented images
    augmented_df = pd.DataFrame(new_rows)

    # Combine original and augmented data
    balanced_df = pd.concat([X_train, augmented_df], ignore_index=True)

    # Print final distribution
    print("\nClass distribution after augmentation:")
    final_percentages = balanced_df['class'].value_counts(normalize=True).apply(lambda x: f"{x:.1%}")
    print(final_percentages)

    return balanced_df

# Run the augmentation
balanced_train_df = augment_underrepresented_classes(X_train)

# Save the balanced dataset information
balanced_train_df.to_csv(DATASET_DIR / 'balanced_train.csv', index=False)

# Print summary of changes
print("\nAugmentation Summary:")
original_dist = X_train['class'].value_counts(normalize=True)
final_dist = balanced_train_df['class'].value_counts(normalize=True)
for class_name in final_dist.index:
    orig_pct = original_dist[class_name] * 100
    final_pct = final_dist[class_name] * 100
    if orig_pct < 5:
        print(f"{class_name}: {orig_pct:.1f}% → {final_pct:.1f}% of total samples")

Minimum target samples per class (5%): 1538


100%|██████████| 16/16 [32:14<00:00, 120.93s/it]



Class distribution after augmentation:
class
Corn_Cercospora_Leaf_Spot    15.3%
Tomato_Septoria              15.2%
Tomato_Late_Blight            9.4%
Corn_Streak                   7.4%
Tomato_Healthy                6.6%
Pepper_Septoria               5.0%
Pepper_Leaf_Mosaic            4.9%
Tomato_Early_Blight           4.8%
Pepper_Bacterial_Spot         4.6%
Corn_Common_Rust              4.6%
Pepper_Leaf_Curl              3.8%
Tomato_Fusarium               3.6%
Corn_Healthy                  3.6%
Pepper_Healthy                1.9%
Pepper_Late_Blight            1.7%
Pepper_Leaf_Blight            1.4%
Tomato_Bacterial_Spot         1.2%
Pepper_Cercospora             1.1%
Pepper_Fusarium               1.1%
Tomato_Leaf_Curl              1.1%
Corn_Northern_Leaf_Blight     1.1%
Tomato_Mosaic                 0.4%
Pepper_Early_Blight           0.3%
Name: proportion, dtype: object

Augmentation Summary:
Tomato_Early_Blight: 4.8% → 4.8% of total samples
Pepper_Bacterial_Spot: 4.8% → 4.6% of total 

In [18]:
import multiprocessing
from pathlib import Path
import numpy as np
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd

# Function to convert the bboxes to YOLO format
def convert_to_yolo(bbox, width, height):
    ymin, xmin, ymax, xmax = bbox['ymin'], bbox['xmin'], bbox['ymax'], bbox['xmax']
    class_id = bbox['class_id']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    return f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
# Top-level function to save annotations for a single image
def save_yolo_annotations_task(task):
    image_path, bboxes, output_dir = task
    try:
        img = np.array(Image.open(str(image_path)))
        height, width, _ = img.shape
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return

    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"
    with open(label_file, 'w') as f:
        for bbox in bboxes:
            annotation = convert_to_yolo(bbox, width, height)
            f.write(f"{annotation}\n")

# Function to clear output directory
def clear_output_dir(output_dir):
    if Path(output_dir).exists():
        shutil.rmtree(output_dir)
    Path(output_dir).mkdir(parents=True, exist_ok=True)
# Function to process the dataset and save annotations
def process_dataset(dataframe, output_dir):
    # Clear the output directory to prevent duplicate annotations
    clear_output_dir(output_dir)

    # Group the DataFrame by 'image_path'
    grouped = dataframe.groupby('image_path')
    tasks = [(image_path, group.to_dict('records'), output_dir) for image_path, group in grouped]

    # Use multiprocessing Pool to process tasks
    with multiprocessing.Pool() as pool:
        list(tqdm(pool.imap_unordered(save_yolo_annotations_task, tasks), total=len(tasks)))


# Save train and validation labels to their respective dirs
process_dataset(X_train, TRAIN_LABELS_DIR)
process_dataset(X_val, VAL_LABELS_DIR)

100%|██████████| 1226/1226 [00:41<00:00, 29.32it/s]


In [ ]:

# Train images dir
TRAIN_IMAGES_DIR

PosixPath('/content/drive/MyDrive/ghana_project/datasets/dataset/images/train')

In [15]:
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)

data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'test': str(TEST_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names,
    'path': str(DATASET_DIR)  # Add dataset root path
}

with open('data.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [16]:


# Define the file path
file_path = "/content/drive/MyDrive/ghana_project"

# Create the full path for data.yaml
yaml_path = os.path.join(file_path, 'data.yaml')

class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)

data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'test': str(TEST_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names,
    'path': str(DATASET_DIR)  # Add dataset root path
}

# Create directory if it doesn't exist
os.makedirs(file_path, exist_ok=True)

# Save the yaml file in the specified path
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [19]:
# Plot some images and their bboxes to ensure the conversion was done correctly
def load_annotations(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    boxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        boxes.append((class_id, x_center, y_center, width, height))
    return boxes

# Function to plot an image with its bounding boxes
def plot_image_with_boxes(image_path, boxes):
    # Load the image
    image = np.array(Image.open(str(image_path)))


    # Get image dimensions
    h, w, _ = image.shape

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

    # Plot each bounding box
    for box in boxes:
        class_id, x_center, y_center, width, height = box
        # Convert YOLO format to corner coordinates
        xmin = int((x_center - width / 2) * w)
        ymin = int((y_center - height / 2) * h)
        xmax = int((x_center + width / 2) * w)
        ymax = int((y_center + height / 2) * h)

        # Draw the bounding box
        plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                          edgecolor='red', facecolor='none', linewidth=2))
        plt.text(xmin, ymin - 10, f'Class {int(class_id)}', color='red', fontsize=8, weight='bold')

    plt.axis('off')
    plt.show()

# Directories for images and labels
IMAGE_DIR = TRAIN_IMAGES_DIR
LABEL_DIR = TRAIN_LABELS_DIR

# Plot a few images with their annotations
for image_name in os.listdir(IMAGE_DIR)[:5]:
    image_path = IMAGE_DIR / image_name
    label_path = LABEL_DIR / (image_name.replace('.jpg', '.txt').replace('.png', '.txt'))

    if label_path.exists():
        boxes = load_annotations(label_path)
        print(f"Plotting {image_name} with {len(boxes)} bounding boxes.")
        plot_image_with_boxes(image_path, boxes)
    else:
        print(f"No annotations found for {image_name}.")


Output hidden; open in https://colab.research.google.com to view.

In [20]:
import os
from ultralytics import YOLO

# Initialize the model
model = YOLO('yolov10x.pt')

# Training configuration with supported augmentations
cfg = {
    # Basic training parameters
    'data': 'data.yaml',
    'epochs': 100,
    'imgsz': 1024,
    'batch': 8,
    'device': 0,
    'patience': 5,
    'val': True,

    # Augmentation parameters (supported by YOLO)
    'augment': True,     # Enable built-in augmentations
    'mixup': 0.3,        # Mixup augmentation
    'mosaic': 0.7,       # Mosaic augmentation
    'degrees': 20.0,     # Maximum rotation degrees
    'translate': 0.2,    # Translation
    'scale': 0.5,        # Scaling
    'shear': 10.0,       # Shearing
    'flipud': 0.3,       # Vertical flip probability
    'fliplr': 0.5,       # Horizontal flip probability
    'hsv_h': 0.015,      # HSV Hue augmentation
    'hsv_s': 0.7,        # HSV Saturation augmentation
    'hsv_v': 0.4,        # HSV Value augmentation

    # Save settings
    'project': '/content/drive/MyDrive/ghana_project',
    'name': 'balanced_model',
    'save': True,
    'save_period': 10    # Save checkpoint every 10 epochs
}

# Train with augmentations
results = model.train(**cfg)

# Save the final model
final_model_path = os.path.join('/content/drive/MyDrive/ghana_project', 'best.pt')
model.save(final_model_path)

100%|██████████| 61.4M/61.4M [00:00<00:00, 145MB/s]


Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov10x.pt, data=data.yaml, epochs=100, time=None, patience=5, batch=8, imgsz=1024, save=True, save_period=10, cache=False, device=0, workers=8, project=/content/drive/MyDrive/ghana_project, name=balanced_model3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]


Overriding model.yaml nc=80 with nc=23

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.2MB/s]


...

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  99%|█████████▊| 76/77 [00:17<00:00,  3.93it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:17<00:00,  4.43it/s]


                   all       1226      10252      0.472       0.34      0.297      0.131
Corn_Cercospora_Leaf_Spot        202       1600      0.359      0.592      0.468       0.19
      Corn_Common_Rust         79        442      0.374      0.287      0.266     0.0965
          Corn_Healthy         63        415      0.269      0.381      0.252     0.0905
Corn_Northern_Leaf_Blight         12         25      0.438       0.32      0.319      0.134
           Corn_Streak         87        804      0.346      0.407      0.319      0.105
 Pepper_Bacterial_Spot        132        476      0.515      0.112      0.235     0.0655
     Pepper_Cercospora         41        121      0.505      0.135      0.244     0.0939
   Pepper_Early_Blight          2         15          1          0          0          0
       Pepper_Fusarium         52        123       0.44      0.634      0.549      0.173
        Pepper_Healthy         68        184      0.461      0.715       0.56      0.237
    Pepper_Late

In [21]:
# Validate the model on the validation set
model = YOLO('/content/drive/MyDrive/ghana_project/balanced_model3/weights/best.pt')
results = model.val()

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv10x summary (fused): 503 layers, 31,628,378 parameters, 0 gradients, 170.0 GFLOPs


val: Scanning /content/drive/MyDrive/ghana_project/datasets/dataset/labels/val.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:29<00:00,  2.57it/s]


                   all       1226      10252      0.474      0.339      0.297      0.132
Corn_Cercospora_Leaf_Spot        202       1600      0.361      0.589      0.466      0.189
      Corn_Common_Rust         79        442      0.373      0.281      0.265     0.0969
          Corn_Healthy         63        415      0.272      0.381      0.252     0.0906
Corn_Northern_Leaf_Blight         12         25      0.438       0.32      0.319      0.134
           Corn_Streak         87        804      0.347      0.404      0.319      0.106
 Pepper_Bacterial_Spot        132        476      0.522      0.109      0.236      0.066
     Pepper_Cercospora         41        121        0.5      0.132      0.244     0.0941
   Pepper_Early_Blight          2         15          1          0          0          0
       Pepper_Fusarium         52        123      0.442      0.634      0.548      0.174
        Pepper_Healthy         68        184      0.463      0.712      0.558      0.237
    Pepper_Late

In [22]:
# Validate the model on the validation set
model = YOLO('/content/drive/MyDrive/ghana_project/balanced_model3/weights/last.pt')
results = model.val()

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv10x summary (fused): 503 layers, 31,628,378 parameters, 0 gradients, 170.0 GFLOPs


val: Scanning /content/drive/MyDrive/ghana_project/datasets/dataset/labels/val.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:21<00:00,  3.51it/s]


                   all       1226      10252      0.464      0.349      0.298       0.13
Corn_Cercospora_Leaf_Spot        202       1600      0.315      0.654      0.484      0.199
      Corn_Common_Rust         79        442      0.369      0.314      0.268     0.0947
          Corn_Healthy         63        415      0.355      0.345      0.287     0.0993
Corn_Northern_Leaf_Blight         12         25      0.163       0.36      0.139     0.0662
           Corn_Streak         87        804      0.337      0.454      0.326      0.106
 Pepper_Bacterial_Spot        132        476      0.417      0.176      0.227     0.0655
     Pepper_Cercospora         41        121      0.313      0.264      0.214     0.0951
   Pepper_Early_Blight          2         15          1          0     0.0274     0.0156
       Pepper_Fusarium         52        123      0.407      0.642      0.541      0.163
        Pepper_Healthy         68        184      0.526      0.696      0.586      0.245
    Pepper_Late

In [23]:
# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/ghana_project/balanced_model3/weights/last.pt')

# Path to the test images directory
test_dir_path = '/content/drive/MyDrive/ghana_project/datasets/dataset/images/test'

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Iterate through each image in the directory
for image_file in tqdm(image_files):
    # Full path to the image
    img_path = os.path.join(test_dir_path, image_file)

    # Make predictions on the image
    results = model(img_path)

    # Extract bounding boxes, confidence scores, and class labels
    boxes = results[0].boxes.xyxy.tolist()  # Bounding boxes in xyxy format
    classes = results[0].boxes.cls.tolist()  # Class indices
    confidences = results[0].boxes.conf.tolist()  # Confidence scores
    names = results[0].names  # Class names dictionary


    # Iterate through the results for this image
    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box
        detected_class = names[int(cls)]  # Get the class name from the names dictionary

        # Add the result to the all_data list
        all_data.append({
            'Image_ID': image_file,
            'class': detected_class,
            'confidence': conf,
            'ymin': y1,
            'xmin': x1,
            'ymax': y2,
            'xmax': x2
        })

# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

  0%|          | 0/2101 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_jj91ad.jpg: 768x1024 (no detections), 64.6ms
Speed: 6.6ms preprocess, 64.6ms inference, 0.6ms postprocess per image at shape (1, 3, 768, 1024)


  0%|          | 1/2101 [00:01<50:32,  1.44s/it]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_jk1pii.jpg: 704x1024 4 Tomato_Early_Blights, 73.4ms
Speed: 5.4ms preprocess, 73.4ms inference, 0.6ms postprocess per image at shape (1, 3, 704, 1024)


  0%|          | 2/2101 [00:01<30:03,  1.16it/s]

...

100%|█████████▉| 2099/2101 [06:00<00:00,  7.02it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2rstzs.jpg: 768x1024 1 Pepper_Bacterial_Spot, 20.9ms
Speed: 6.1ms preprocess, 20.9ms inference, 0.5ms postprocess per image at shape (1, 3, 768, 1024)


100%|█████████▉| 2100/2101 [06:00<00:00,  7.27it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2sg62b.jpg: 480x1024 2 Corn_Healthys, 21.0ms
Speed: 4.1ms preprocess, 21.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 1024)


100%|██████████| 2101/2101 [06:00<00:00,  5.83it/s]


In [24]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_jk1pii.jpg,Tomato_Early_Blight,0.709827,326.170258,1814.728027,441.049103,1902.802246
1,id_jk1pii.jpg,Tomato_Early_Blight,0.703605,624.191162,876.092041,763.785828,973.352539
2,id_jk1pii.jpg,Tomato_Early_Blight,0.509028,587.087769,772.248291,649.085632,819.555176
3,id_jk1pii.jpg,Tomato_Early_Blight,0.321725,189.567963,781.766174,239.366608,835.058655
4,id_jk1u1s.jpg,Corn_Cercospora_Leaf_Spot,0.594625,1084.118164,1720.048828,1196.838379,1879.260742


In [25]:
# Save to the same project directory in your Drive
submission_path = '/content/drive/MyDrive/ghana_project/benchmark_submission.csv'
sub.to_csv(submission_path, index=False)
print(f"Submission saved to: {submission_path}")


Submission saved to: /content/drive/MyDrive/ghana_project/benchmark_submission.csv


In [27]:
sub['Image_ID'].value_counts()

,count
Image_ID,
id_dozhbt.jpg,36
id_sjo7av.jpg,33
id_mkp6a7.jpg,33
id_hz628c.jpg,33
id_1ecbrb.jpg,29
...,...
id_vxst38.jpg,1
id_w1q9ql.jpg,1
id_umhp87.jpg,1


In [30]:
import os
import random
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np

# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/ghana_project/balanced_model3/weights/last.pt')

# Path to the test images directory
test_dir_path = '/content/drive/MyDrive/ghana_project/datasets/dataset/images/test'

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

def generate_random_box(img_size=(640, 640)):
    """Generate a random bounding box within image dimensions."""
    # Generate random box with reasonable size (20-40% of image size)
    width = random.uniform(0.2, 0.4) * img_size[0]
    height = random.uniform(0.2, 0.4) * img_size[1]

    # Generate random position
    x1 = random.uniform(0, img_size[0] - width)
    y1 = random.uniform(0, img_size[1] - height)

    x2 = x1 + width
    y2 = y1 + height

    return [x1, y1, x2, y2]

# Set of processed images to track which ones have predictions
processed_images = set()

# Iterate through each image in the directory
for image_file in tqdm(image_files):
    # Full path to the image
    img_path = os.path.join(test_dir_path, image_file)

    # Make predictions on the image
    results = model(img_path)

    # Extract bounding boxes, confidence scores, and class labels
    boxes = results[0].boxes.xyxy.tolist()  # Bounding boxes in xyxy format
    classes = results[0].boxes.cls.tolist()  # Class indices
    confidences = results[0].boxes.conf.tolist()  # Confidence scores
    names = results[0].names  # Class names dictionary

    # If predictions were found, add them to all_data
    if len(boxes) > 0:
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[int(cls)]

            all_data.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })
        processed_images.add(image_file)

    # If no predictions were found, generate random box and class
    else:
        # Generate random box
        random_box = generate_random_box()

        # Select random class from available classes
        random_class = random.choice(list(names.values()))

        # Generate random confidence between 0.1 and 0.3 (low confidence for random predictions)
        random_conf = random.uniform(0.1, 0.3)

        all_data.append({
            'Image_ID': image_file,
            'class': random_class,
            'confidence': random_conf,
            'ymin': random_box[1],
            'xmin': random_box[0],
            'ymax': random_box[3],
            'xmax': random_box[2]
        })
        processed_images.add(image_file)

# Double check if any images were missed
for image_file in image_files:
    if image_file not in processed_images:
        # Generate random prediction for missed images
        random_box = generate_random_box()
        random_class = random.choice(list(names.values()))
        random_conf = random.uniform(0.1, 0.3)

        all_data.append({
            'Image_ID': image_file,
            'class': random_class,
            'confidence': random_conf,
            'ymin': random_box[1],
            'xmin': random_box[0],
            'ymax': random_box[3],
            'xmax': random_box[2]
        })

# Convert the list to a DataFrame
sub2 = pd.DataFrame(all_data)

# Verify that all images have predictions
print(f"Total number of images: {len(image_files)}")
print(f"Number of unique images in predictions: {len(sub2['Image_ID'].unique())}")
print(f"All images have predictions: {len(image_files) == len(sub2['Image_ID'].unique())}")

  0%|          | 0/2101 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_jj91ad.jpg: 768x1024 (no detections), 21.3ms
Speed: 6.1ms preprocess, 21.3ms inference, 0.4ms postprocess per image at shape (1, 3, 768, 1024)


  0%|          | 1/2101 [00:00<23:22,  1.50it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_jk1pii.jpg: 704x1024 4 Tomato_Early_Blights, 25.3ms
Speed: 5.0ms preprocess, 25.3ms inference, 0.7ms postprocess per image at shape (1, 3, 704, 1024)

image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_jk1u1s.jpg: 480x1024 9 Corn_Cercospora_Leaf_Spots, 20.1ms
Speed: 3.8ms preprocess, 20.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 1024)


  0%|          | 3/2101 [00:00<07:39,  4.56it/s]

...


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2q7lnt.jpg: 480x1024 4 Corn_Cercospora_Leaf_Spots, 3 Corn_Common_Rusts, 20.5ms
Speed: 3.9ms preprocess, 20.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 1024)

image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2rl9gr.jpg: 768x1024 14 Corn_Northern_Leaf_Blights, 20.9ms
Speed: 6.1ms preprocess, 20.9ms inference, 0.5ms postprocess per image at shape (1, 3, 768, 1024)


100%|█████████▉| 2099/2101 [03:13<00:00, 10.83it/s]


image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2rstzs.jpg: 768x1024 1 Pepper_Bacterial_Spot, 20.3ms
Speed: 6.0ms preprocess, 20.3ms inference, 0.5ms postprocess per image at shape (1, 3, 768, 1024)

image 1/1 /content/drive/MyDrive/ghana_project/datasets/dataset/images/test/id_2sg62b.jpg: 480x1024 2 Corn_Healthys, 21.2ms
Speed: 3.8ms preprocess, 21.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 1024)


100%|██████████| 2101/2101 [03:13<00:00, 10.87it/s]

Total number of images: 2101
Number of unique images in predictions: 2101
All images have predictions: True


In [31]:
sub2.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_jj91ad.jpg,Tomato_Early_Blight,0.197186,107.445749,169.874751,332.463912,405.960748
1,id_jk1pii.jpg,Tomato_Early_Blight,0.709827,326.170258,1814.728027,441.049103,1902.802246
2,id_jk1pii.jpg,Tomato_Early_Blight,0.703605,624.191162,876.092041,763.785828,973.352539
3,id_jk1pii.jpg,Tomato_Early_Blight,0.509028,587.087769,772.248291,649.085632,819.555176
4,id_jk1pii.jpg,Tomato_Early_Blight,0.321725,189.567963,781.766174,239.366608,835.058655


In [32]:
# Save to the same project directory in your Drive
submission_path = '/content/drive/MyDrive/ghana_project/prediction_submission.csv'
sub2.to_csv(submission_path, index=False)
print(f"Submission saved to: {submission_path}")


Submission saved to: /content/drive/MyDrive/ghana_project/prediction_submission.csv
